In [474]:
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import statsmodels.api as sm
import statsmodels.formula.api as smf
plt.style.use('seaborn-whitegrid')
from sklearn import datasets
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

In [475]:
# magic to reload modules automatically
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [476]:
#!pip install stargazer
#conda install -c conda-forge statsmodels

In [477]:
rt = pd.read_excel('rt.xlsx')
msci = pd.read_excel('data/KP.xlsx', skiprows=2)
esg_esg = pd.read_excel('esg_esg.xlsx')
it = pd.read_excel('it.xlsx')
ff = pd.read_excel('ff.xlsx')
esg_esg.rename(columns={'Unnamed: 0':'scores'}, inplace=True)
rt = rt.replace(0, np.nan)
rt.dropna(how='all', axis='index', thresh=4, inplace=True)
rt = rt.replace(np.nan, 0)

In [478]:
pd.set_option('display.max_columns',125) # I want to see all the columns
pd.set_option('display.max_rows',600) 

In [479]:
it = it.loc[:,['Unnamed: 0.1','DAX Index','OMXS30B Index','OMXC25 Index','OMX Index']]

for i in it.columns[1:]:
    it.loc[:,i] = it.loc[:,i].pct_change(1)*100
it.drop(index=[0], inplace=True)
it.rename(columns={'Unnamed: 0.1' : 'date'}, inplace=True)
rt.drop(columns='Unnamed: 0', inplace=True)
ff.drop(columns='Unnamed: 0', inplace=True)
rt = rt.loc[:,['date','port_low','port_mid','port_high']]

In [480]:
msci.drop(index=[0,1,2], inplace=True)
msci.rename(columns = {'Unnamed: 0' : 'date','Unnamed: 1' : 'MSCI_World', 'Unnamed: 2' : 'SPX', 'Unnamed: 3' : 'MSCI_Eur'}, inplace=True)
msci['date'] = pd.to_datetime(msci['date'], format='%Y%m%d')
for i in msci.columns[1:]:
    msci.loc[:,i] = msci.loc[:,i].pct_change(1)*100
msci.reset_index(inplace = True, drop = True)
msci.drop(index=[0], inplace=True)

In [481]:
rt = rt.merge(ff,on=['date'], how='left')
rt = rt.merge(it,on=['date'], how='left')
rt = rt.merge(msci,on=['date'], how='left')
pd.set_option("display.precision", 3)
#rt['RF'] = rt['RF']*100
rt.rename(columns = {'DAX Index' : 'DAX','OMXS30B Index' : 'S30','OMXC25 Index' : 'C25',
                     'OMX Index' : 'OMX','Mkt-RF' : 'MktRF'}, inplace=True)

In [482]:
filt = (rt['date'] >= pd.to_datetime('2013-01-01')) & (rt['date'] <= pd.to_datetime('2022-02-28')) 
rt = rt[filt]
rt.dropna(how='all', axis='index', thresh=5, inplace=True)

In [483]:
rt['MSCI_World'] = rt['MSCI_World'] - rt['RF']

In [484]:
filt = rt[['port_low','port_mid','port_high']].describe()
filt
#print(filt.to_latex(index = True, multirow = True))

,port_low,port_mid,port_high
count,2273.000,2273.000,2273.000
mean,0.078,0.041,0.048
std,1.049,1.023,1.076
min,-10.321,-10.736,-11.413
25%,-0.403,-0.429,-0.424
50%,0.124,0.077,0.084
75%,0.613,0.576,0.568
max,7.275,6.682,8.876


In [485]:
ame = ['SPX','MSCI_Eur']
rt.drop(columns=ame, inplace = True)
filt2 = rt.describe()
#print(filt2.to_latex(index = True, multirow = True))

\begin{tabular}{lrrrrrrrrrrrrrr}
\toprule
{} &  port\_low &  port\_mid &  port\_high &     MktRF &       SMB &       HML &       RMW &       CMA &        RF &       DAX &       S30 &       C25 &       OMX &  MSCI\_World \\
\midrule
count &  2273.000 &  2273.000 &   2273.000 &  2273.000 &  2273.000 &  2273.000 &  2273.000 &  2273.000 &  2273.000 &  2273.000 &  2273.000 &  2273.000 &  2273.000 &    2273.000 \\
mean  &     0.078 &     0.041 &      0.048 &     0.029 &     0.008 &    -0.007 &     0.014 &    -0.009 &     0.002 &     0.034 &     0.031 &     0.054 &     0.031 &       0.032 \\
std   &     1.049 &     1.023 &      1.076 &     1.015 &     0.420 &     0.537 &     0.303 &     0.296 &     0.004 &     1.218 &     1.113 &     1.053 &     1.113 &       0.893 \\
min   &   -10.321 &   -10.736 &    -11.413 &   -12.010 &    -3.320 &    -3.030 &    -1.740 &    -1.740 &     0.000 &   -12.239 &   -10.571 &    -8.029 &   -10.571 &      -9.925 \\
25\%   &    -0.403 &    -0.429 &     -0.424 &   

In [486]:
filt2

,port_low,port_mid,port_high,MktRF,SMB,HML,RMW,CMA,RF,DAX,S30,C25,OMX,MSCI_World
count,2273.000,2273.000,2273.000,2273.000,2273.000,2273.000,2273.000,2273.000,2273.000,2273.000,2273.000,2273.000,2273.000,2273.000
mean,0.078,0.041,0.048,0.029,0.008,-0.007,0.014,-0.009,0.002,0.034,0.031,0.054,0.031,0.032
std,1.049,1.023,1.076,1.015,0.420,0.537,0.303,0.296,0.004,1.218,1.113,1.053,1.113,0.893
min,-10.321,-10.736,-11.413,-12.010,-3.320,-3.030,-1.740,-1.740,0.000,-12.239,-10.571,-8.029,-10.571,-9.925
25%,-0.403,-0.429,-0.424,-0.440,-0.230,-0.280,-0.160,-0.190,0.000,-0.491,-0.533,-0.514,-0.532,-0.297
50%,0.124,0.077,0.084,0.070,0.010,-0.030,0.030,-0.010,0.000,0.070,0.058,0.063,0.058,0.065
75%,0.613,0.576,0.568,0.550,0.250,0.260,0.190,0.160,0.000,0.641,0.639,0.673,0.639,0.449
max,7.275,6.682,8.876,8.540,1.860,4.380,2.690,1.310,0.010,10.976,7.089,5.073,7.089,8.760


In [487]:
model1 = smf.ols(formula='port_low ~ MktRF', data=rt).fit() 
model11 = smf.ols(formula='port_low ~ MktRF + SMB + HML', data=rt).fit() 
model111 = smf.ols(formula='port_low ~ MktRF + SMB + HML + RMW + CMA', data=rt).fit() 

In [488]:
model2 = smf.ols(formula='port_mid ~ MktRF', data=rt).fit() 
model22 = smf.ols(formula='port_mid ~ MktRF + SMB + HML', data=rt).fit() 
model222 = smf.ols(formula='port_mid ~ MktRF + SMB + HML + RMW + CMA', data=rt).fit() 

In [489]:
model3 = smf.ols(formula='port_high ~ MktRF', data=rt).fit() 
model33 = smf.ols(formula='port_high ~ MktRF + SMB + HML', data=rt).fit() 
model333 = smf.ols(formula='port_high ~ MktRF + SMB + HML + RMW + CMA', data=rt).fit() 
#print(model3.summary())
#trend = model3.params[0] + model3.params[1]* rt['port_high']
#plt.scatter(rt['port_high'],rt['MktRF'])
#plt.plot(trend, rt['port_high'], 'r');

In [490]:
stargazer = Stargazer([model1, model2, model3, model11, model22, model22, model111, model222, model333])
stargazer.custom_columns(['CAMP low', 'CAMP mid', 'CAMP high', 'FF3 low', 'FF3 mid', 'FF3 high', 'FF5 low', 'FF5 mid', 'FF5 high'], [1, 1, 1, 1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
stargazer.covariate_order(['Intercept', 'MktRF', 'SMB', 'HML', 'RMW', 'CMA'])
stargazer.show_degrees_of_freedom(False)
stargazer.rename_covariates({'Intercept': 'alpha','MktRF' : 'beta_1','SMB' : 'beta_2 (SMB)', 'HML' : 'beta_3 (HML)',
                          'RMW' : 'beta_4 (RMW)', 'CMA' : 'beta_5 (CMA)'}) #if you want to change names
HTML(stargazer.render_html())
# Use this guide to addit the table
#print(stargazer.render_latex()) #compile it in LaTeX instead of HTML

In [491]:
def SR(m):
    SR = (m.mean()/m.std())*252**.5
    return SR
sharpe_ratio = []
for i in filt.columns[:3]:
    sharpe_ratio.append(SR(rt[i]))

In [492]:
betas = [model1.params[1], model2.params[1], model3.params[1]]
def TR(m):
    for j in betas:
        TR = (m.mean()/ j )*252**.5
    return TR
treynor_ratio = []
for i in filt.columns[:3]:
    treynor_ratio.append(TR(rt[i]))

In [493]:
betas

[0.8249481942682456, 0.8457153222240437, 0.9075540812200436]

In [494]:
sharpe_ratio

[1.1784773031043825, 0.6308812742106769, 0.7106697872351895]

In [495]:
treynor_ratio

[1.3616931455181347, 0.711237840381233, 0.8426392098475165]

In [496]:
model1 = smf.ols(formula='port_low ~ MSCI_World', data=rt).fit() 
model11 = smf.ols(formula='port_low ~ MSCI_World + SMB + HML', data=rt).fit() 
model111 = smf.ols(formula='port_low ~ MSCI_World + SMB + HML + RMW + CMA', data=rt).fit() 

model2 = smf.ols(formula='port_mid ~ MSCI_World', data=rt).fit() 
model22 = smf.ols(formula='port_mid ~ MSCI_World + SMB + HML', data=rt).fit() 
model222 = smf.ols(formula='port_mid ~ MSCI_World + SMB + HML + RMW + CMA', data=rt).fit() 

model3 = smf.ols(formula='port_high ~ MSCI_World', data=rt).fit() 
model33 = smf.ols(formula='port_high ~ MSCI_World + SMB + HML', data=rt).fit() 
model333 = smf.ols(formula='port_high ~ MSCI_World + SMB + HML + RMW + CMA', data=rt).fit() 
#print(model3.summary())
#trend = model3.params[0] + model3.params[1]* rt['port_high']
#plt.scatter(rt['port_high'],rt['MktRF'])
#plt.plot(trend, rt['port_high'], 'r');

stargazer = Stargazer([model1, model2, model3, model11, model22, model22, model111, model222, model333])
stargazer.custom_columns(['CAMP low', 'CAMP mid', 'CAMP high', 'FF3 low', 'FF3 mid', 'FF3 high', 'FF5 low', 'FF5 mid', 'FF5 high'], [1, 1, 1, 1, 1, 1, 1, 1, 1])
stargazer.show_model_numbers(False)
stargazer.covariate_order(['Intercept', 'MSCI_World', 'SMB', 'HML', 'RMW', 'CMA'])
stargazer.show_degrees_of_freedom(False)
stargazer.rename_covariates({'Intercept': 'alpha','MSCI_World' : 'beta_1','SMB' : 'beta_2 (SMB)', 'HML' : 'beta_3 (HML)',
                          'RMW' : 'beta_4 (RMW)', 'CMA' : 'beta_5 (CMA)'}) #if you want to change names
#HTML(stargazer.render_html())
# Use this guide to addit the table
#print(stargazer.render_latex()) #compile it in LaTeX instead of HTML

In [497]:
filt3 = rt[['port_low','port_mid','port_high']]
col = filt3.columns.tolist()
mean_list = (rt[['port_low','port_mid','port_high']].mean() * 252).tolist()
sd_list = (rt[['port_low','port_mid','port_high']].std() * 252**0.5).tolist()

In [498]:
stra = zip(mean_list, sd_list, sharpe_ratio, treynor_ratio)
res = dict(zip(col, stra))
res = pd.DataFrame(res, index = ['Mean','SD','SR','TR'])
pd.set_option("display.precision", 2)

In [499]:
res

,port_low,port_mid,port_high
Mean,19.62,10.25,12.14
SD,16.65,16.24,17.08
SR,1.18,0.63,0.71
TR,1.36,0.71,0.84


In [500]:
#print(res.to_latex(index = True, multirow = True))

\begin{tabular}{lrrr}
\toprule
{} &  port\_low &  port\_mid &  port\_high \\
\midrule
Mean &     19.62 &     10.25 &      12.14 \\
SD   &     16.65 &     16.24 &      17.08 \\
SR   &      1.18 &      0.63 &       0.71 \\
TR   &      1.36 &      0.71 &       0.84 \\
\bottomrule
\end{tabular}

